# DATA MINING PROJECT: Analysis of a Supermarket’s Customers
## 1.2) Data Preparation
### *Antonio Strippoli, Valerio Mariani*

In [ ]:
%matplotlib inline
import os
import calendar
import numpy as np
import pandas as pd
import seaborn as sn
from math import log, ceil
import matplotlib.pyplot as plt

pd.set_option('mode.chained_assignment', None)

In [ ]:
def plot(ax, folder="cdf_plots", filename="", figsize=(6.4, 4.8)):
    fig = plt.gcf()
    fig.set_size_inches(*figsize)
    plt.tight_layout()
    if filename:
        path = os.path.join("..", "report", "imgs", folder)
        if not os.path.exists(path):
            os.mkdir(path)
        plt.savefig(os.path.join(path, filename))
    plt.show()
    plt.close()

In [ ]:
# Load the secondary data from the given file
df = pd.read_csv('customer_supermarket_2.csv', index_col=0, parse_dates=["PurchaseDate"])

### Create a new dataset with a profilation of each customer.

In [ ]:
# Recency
recency = lambda g: (df['PurchaseDate'].max() - g['PurchaseDate'].max()).days
# Frequency
frequency = lambda g: g['BasketID'].nunique()
# Total money spent
monetary = lambda g: round( sum( g["Sale"]*g["Qta"] ), 2)
# Total purchased items
tot_items = lambda g: sum( g["Qta"] )
# Maximum number of purchased items in a shopping session
max_items = lambda g: max( [ sum( g1[1]["Qta"] ) for g1 in g.groupby("BasketID") ] )
# Medium object in basket
mean_items = lambda g: int( np.mean( [ sum( g1[1]["Qta"] ) for g1 in g.groupby("BasketID") ] ))
# Number of distinct items
unique_items = lambda g: g["ProdID"].nunique()
# Preferred item
preferred_item = lambda g: g.groupby('ProdID').agg({'Qta':'sum'}).idxmax()[0]
# Max amount for a basket
max_sale = lambda g: round( max( [ sum( g1[1]["Sale"]*g1[1]["Qta"] ) for g1 in g.groupby("BasketID") ] ), 2)
# Medium amount for a basket
mean_sale = lambda g: round( np.mean( [ sum( g1[1]["Sale"]*g1[1]["Qta"] ) for g1 in g.groupby( "BasketID" ) ] ), 2)
# Medium amount paid for an item
mean_item_sale = lambda g: round( np.mean( g["Sale"].unique() ), 2)
# Entropy
def entropy(g, attr1, attr2, groupby, op):
    l = g[[attr1, attr2]].groupby(groupby).agg(op)
    m = l.values.sum()
    e = -sum( [ (mi/m)*log((mi/m), 2) for mi in l.values.flatten() ] )
    return round(e, 2)
# Weekday preference
weekday_pref = lambda g: int(pd.Series(g['PurchaseDate'].unique()).apply(lambda x: x.weekday()).mean())
# Number of week of the month preference
weekmonth_pref = lambda g: int(pd.Series(g['PurchaseDate'].unique()).apply( lambda x: int(ceil(x.day/7.0)) ).mean())
# Main country
main_country = lambda g: g[['BasketID','CustomerCountry']].groupby('CustomerCountry').nunique().idxmax()[0]

In [ ]:
groups = df[df["Qta"]>0].groupby("CustomerID")
cdf = pd.DataFrame(data=np.array( [
    [
    group[0],
    recency(group[1]),
    frequency(group[1]),
    monetary(group[1]),
    tot_items(group[1]),
    max_items(group[1]),
    mean_items(group[1]),
    unique_items(group[1]),
    preferred_item(group[1]),
    max_sale(group[1]),
    mean_sale(group[1]),
    mean_item_sale(group[1]),
    entropy(group[1], "ProdID", 'Qta', 'ProdID', 'sum'),
    entropy(group[1], "ProdID", 'Sale', 'ProdID', 'sum'),
    weekday_pref(group[1]),
    weekmonth_pref(group[1]),
    main_country(group[1])
    ] for group in groups
] ), columns=["CustomerID","Recency","Frequency","Monetary","TotItems","MaxItems","MeanItems","UniqueItems","PrefItem","MaxSale","MeanSale","MeanItemSale","E-Qta","E-Sale","WeekDayPref","WeekMonthPref","MainCountry"] )
cdf.set_index('CustomerID', inplace=True)

# Workaround for Pandas' bug (not able to convert to correct dtypes)
# cdf.convert_dtypes()
cdf.to_csv("customer_profilation.csv")
cdf = pd.read_csv("customer_profilation.csv", index_col=0)

# calculate percentage of returned item for customer
groups = df[ (df["Qta"]<0) & ~(df["ProdID"].isin(['M', 'D', 'BANK CHARGES'])) ][['CustomerID','Qta']].groupby("CustomerID").agg('sum')
cdf['PReturn'] = pd.Series(
    [ round(-groups.loc[i]['Qta']/cdf.loc[i]['TotItems']*100, 2) if i in groups.index else 0 for i in cdf.index ],
    dtype='float64',
    index=cdf.index
)

print("N. ENTRIES:", len(cdf))
cdf.to_csv("customer_profilation.csv")

### Data Quality

In [ ]:
cdf = pd.read_csv("customer_profilation.csv", index_col=0)

cdf.info()
cdf.describe()

In [ ]:
# Outliers in TotItems (users who purchased an abnormal number of items)
cdf_totitems = cdf['TotItems']
plot(cdf_totitems.plot.box(), figsize=(2, 4.8), folder="cdf_outliers", filename="TotItems_BP")
plot(sn.distplot(cdf_totitems[cdf_totitems < 25000], bins=100), folder="cdf_outliers", filename="TotItems_HIST")

# Search for a threshold
plot(cdf_totitems[cdf_totitems < 100000].plot.box(), figsize=(2, 4.8))
plot(cdf_totitems[cdf_totitems < 70000].plot.box(), figsize=(2, 4.8))

cdf = cdf[cdf_totitems < 70000]

In [ ]:
# Outliers in Monetary (users who spent way too much money)
cdf_monetary = cdf['Monetary']
plot(cdf_monetary.plot.box(), figsize=(2, 4.8), folder="cdf_outliers", filename="Monetary_BP")
plot(sn.distplot(cdf_monetary[cdf_monetary < 25000], bins=100), folder="cdf_outliers", filename="Monetary_HIST")

# Search for a threshold
plot(cdf_monetary[cdf_monetary < 80000].plot.box(), figsize=(2, 4.8))

cdf = cdf[cdf_monetary < 80000]

In [ ]:
print("N. ENTRIES:", len(cdf))
cdf.to_csv("customer_profilation.csv")

### Distribution & Statistics

In [ ]:
cdf = pd.read_csv("customer_profilation.csv", index_col=0)

In [ ]:
monetary = cdf[cdf['Monetary']<6000]['Monetary']
print(cdf['Monetary'].describe())
monetary.hist(bins=100)

In [ ]:
# Pandas' scatter matrix
plot(pd.plotting.scatter_matrix(cdf), figsize=(20,20), filename="ScatterMatrix")

In [ ]:
# Distribution of numerical attributes with histograms
plot(cdf.hist(bins=50), figsize=(10,10), filename="Histograms")

# Distribution of numerical attributes with box-plots
plot(cdf.plot.box(), filename="Box_Plots")

# Pairwise correlations with heatmap on correlation matrix
plot(sn.heatmap(cdf.corr(), cmap='coolwarm', annot=True), figsize=(10,10), filename="HeatMap_Correlations")

In [ ]:
cdf['Recency'].describe()

In [ ]:
# Study and save some particular distributions and correlations
plot(sn.distplot(cdf['Recency'], bins=100), filename="Recency_HIST")
plot(cdf['WeekDayPref'].value_counts().sort_index().rename(lambda i: calendar.day_name[i]).plot.bar(), filename="WeekDayPref_HIST")
plot(cdf['WeekMonthPref'].value_counts().sort_index().plot.bar(), filename="WeekMonthPref_HIST")
plot(sn.distplot(cdf['PReturn'], bins=100), filename="PReturn_HIST")
plot(sn.distplot(cdf['E-Qta'], bins=100), filename="EQta_HIST")
plot(sn.distplot(cdf['E-Sale'], bins=100), filename="ESale_HIST")

In [ ]:
# Scatter plot to show correlation of the 2 entropies
plot(cdf.plot.scatter('E-Qta', 'E-Sale', c='UniqueItems', colormap='hot', sharex=False), figsize=(8,6), filename="Entropies")

In [ ]:
# 3D Scatter plot to show correlations between entropies, frequency and unique items
fig = plt.figure(figsize = (10, 7))
ax = plt.axes(projection ="3d")

p = ax.scatter3D(cdf['E-Qta'], cdf['E-Sale'], cdf['Frequency'], c=cdf['UniqueItems'], cmap='hot', s=20)
cbar = fig.colorbar(p)
ax.set_xlabel('E-Qta')
ax.set_ylabel('E-Sale')
ax.set_zlabel('Frequency')
cbar.set_label('UniqueItems')

plot(ax, figsize=(8,6), filename="Entropies_3D")

In [ ]:
# Scatter plot to show correlation of PReturn and Frequency
plot(cdf.plot.scatter('PReturn', 'Frequency', c='Recency', colormap='plasma', sharex=False), figsize=(8,6), filename="PReturn_Frequency")